### Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

### Load the Datasets

In [2]:
test = pd.read_csv("../data/Test_0qrQsBZ.csv")
train = pd.read_csv("../data/Train_SU63ISt.csv")

In [3]:
test_original=test.copy()
train_original=train.copy()

### Understand Data Structure & Content

In [20]:
print("=====Train Data Columns=====")
print(train.columns)
print("\n=====Test Data Columns=====")
print(test.columns)

=====Train Data Columns=====
Index(['ID', 'Datetime', 'Count'], dtype='object')

=====Test Data Columns=====
Index(['ID', 'Datetime'], dtype='object')


In [15]:
print("=====Train data types=====")
print(train.dtypes)
print("\n=====Test data types=====")
print(test.dtypes)

=====Train data types=====
ID           int64
Datetime    object
Count        int64
dtype: object

=====Test data types=====
ID           int64
Datetime    object
dtype: object


In [19]:
print("=====Train Data=====")
print(train.head())
print("\n=====Test Data=====")
print(test.head())

=====Train Data=====
   ID          Datetime  Count
0   0  25-08-2012 00:00      8
1   1  25-08-2012 01:00      2
2   2  25-08-2012 02:00      6
3   3  25-08-2012 03:00      2
4   4  25-08-2012 04:00      2

=====Test Data=====
      ID          Datetime
0  18288  26-09-2014 00:00
1  18289  26-09-2014 01:00
2  18290  26-09-2014 02:00
3  18291  26-09-2014 03:00
4  18292  26-09-2014 04:00


In [18]:
print("=====Train Data size=====")
print(train.shape)
print("\n=====Test Data size=====")
print(test.shape)

=====Train Data size=====
(18288, 3)

=====Test Data size=====
(5112, 2)
